In [1]:
%load_ext autoreload
%autoreload 2
import os
import time
import pickle
import numpy as np
import multiprocessing as mp
from IPython.display import SVG

from pydrake.geometry import MeshcatVisualizer, MeshcatVisualizerParams, Role, StartMeshcat
from pydrake.geometry.optimization import IrisInConfigurationSpace, IrisOptions
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.solvers import MosekSolver
from pydrake.systems.framework import DiagramBuilder

from gcs.linear import LinearGCS
from gcs.rounding import randomForwardPathSearch
from reproduction.prm_comparison.helpers import (
    get_traj_length,
    InverseKinematics,
    make_traj,
    visualize_trajectory
)

from reproduction.util import GcsDir, FindModelFile

mosek_license = MosekSolver.AcquireLicense()

In [2]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
SEED = 0
np.random.seed(SEED)
CORE_CNT = mp.cpu_count() # you may edit this.

In [19]:
meshcat.Delete()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)
parser = Parser(plant)
parser.package_map().Add("gcs", GcsDir())

directives_file = FindModelFile("models/scene.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)
[iiwa, wsg, _,_,_,_,_,_,_,_] = models
iiwa_idx = plant.GetModelInstanceByName("iiwa")
wsg_idx = plant.GetModelInstanceByName("wsg")

plant.Finalize()

meshcat_params = MeshcatVisualizerParams()
meshcat_params.delete_on_initialization_event = False
meshcat_params.role = Role.kIllustration
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat,
                                             meshcat_params)

diagram = builder.Build()

context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = [0, 0.2, 0, -2.1, 0, 1, 1.57]
plant.SetPositions(plant_context, q0)
diagram.ForcedPublish(context)